# Sockets

Once an internet connection has been established, MicroPython uses [sockets](https://docs.micropython.org/en/latest/library/usocket.html) to access resources on the network, just like CPython (and pretty much [all programming languages](https://en.wikipedia.org/wiki/Network_socket)).

Sockets are quite low level; frequently higher level libraries can be used instead. But if you want to write your own webserver, for example, you likely will use sockets.

Examples presented are adapted from the [MicroPython github repository](https://github.com/micropython/micropython/tree/master/examples/network). Check them out for additional information.

## http Client

The code below first looks up the ip address of the server (`google.com`). It then creates a `socket`, connects to it at port 80 and downloads 2000 bytes. 

In [1]:
%connect esp32 -q
import socket

ai = socket.getaddrinfo('google.com', 80)
print("Address information:", ai)
addr = ai[0][-1]

s = socket.socket()
s.connect(addr)
s.write(b"GET / HTTP/1.0\r\n\r\n")

print("\nResponse:")
print(s.read(2000).decode(), "...")

s.close()

Address information: [(2, 1, 0, 'google.com', ('216.58.194.174', 80))]

Response:
HTTP/1.0 200 OK
Date: Thu, 22 Jul 2021 03:08:27 GMT
Expires: -1
Cache-Control: private, max-age=0
Content-Type: text/html; charset=ISO-8859-1
P3P: CP="This is not a P3P policy! See g.co/p3phelp for more info."
Server: gws
X-XSS-Protection: 0
X-Frame-Options: SAMEORIGIN
Set-Cookie: 1P_JAR=2021-07-22-03; expires=Sat, 21-Aug-2021 03:08:27 GMT; path=/; domain=.google.com; Secure
Set-Cookie: NID=219=R1eiu3eJy1QMHo1SQIl73n5yc_rtUafRdRoyA9J3l3t5e_NhZyyIkut1XIGY7qd35-IKc-wW3WNeYC1swiM9dfwDKJoJIyY9NiociLLKP7UIPzdGpfirkxf2_8bNfTsRh4PCKRb3Nk_2FOUyAZYyTuOrFMXkgDq8HwcUWH8zgQE; expires=Fri, 21-Jan-2022 03:08:27 GMT; path=/; domain=.google.com; HttpOnly
Accept-Ranges: none
Vary: Accept-Encoding

<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="en"><head><meta content="Search the world's information, including webpages, images, videos and more. Google has many special features to help you find

The response is quite wordy with embedded graphics meant for visualization in a browser, not parse by a microcontroller. Some sites, e.g. for weather data, can produce simpler responses optimized for parsing by machines.

## http Server

Let's do the opposite and create a simple webserver.

In [1]:
%connect esp32 -q

import socket, network


CONTENT = b"""\
HTTP/1.0 200 OK

Hello #{} from MicroPython!
"""

PORT = 8080


def webserver():
    my_ip = network.WLAN(network.STA_IF).ifconfig()[0]
    s = socket.socket()

    # Binding to all interfaces - server will be accessible to other hosts!
    ai = socket.getaddrinfo("0.0.0.0", PORT)
    addr = ai[0][-1]

    s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    s.bind(addr)
    s.listen(5)
    print("Listening, connect your browser to http://{}:{}/".format(my_ip, PORT))

    try:
        counter = 0
        while True:
            client_sock, client_addr = s.accept()

            print("Request from".format(client_addr))
            req = client_sock.readline()
            print("\nRequest:")
            print(req)
            while True:
                h = client_sock.readline()
                if h == b"" or h == b"\r\n":
                    break
                print(h)
            client_sock.write(CONTENT.format(counter))

            client_sock.close()
            counter += 1
            print()
    finally:
        s.close()

webserver()

Listening, connect your browser to http://10.39.40.135:8080/
Request from
b'GET / HTTP/1.1\r\n'
b'Host: 10.39.40.135:8080\r\n'
b'Connection: keep-alive\r\n'
b'DNT: 1\r\n'
b'Upgrade-Insecure-Requests: 1\r\n'
b'User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.164 Safari/537.36\r\n'
b'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9\r\n'
b'Accept-Encoding: gzip, deflate\r\n'
b'Accept-Language: en-US,en;q=0.9,de-CH;q=0.8,de;q=0.7,fr-FR;q=0.6,fr;q=0.5,zh-HK;q=0.4,zh-CN;q=0.3,zh-TW;q=0.2,zh;q=0.1\r\n'

Request from
b'GET /favicon.ico HTTP/1.1\r\n'
b'Host: 10.39.40.135:8080\r\n'
b'Connection: keep-alive\r\n'
b'User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.164 Safari/537.36\r\n'
b'DNT: 1\r\n'
b'Accept: image/avif,image/webp,image/apng,image/svg+xml,image/*,*/*;q=0.8

Interrupted


Click the `http` link above to open a browser window. Notice two things:

1. The hello counter increases by two everytime you refresh the page in the browser. The reason is that the browser (at least mine) make two requests each time it loads the page.

2. The browser sends lots of data with each request. The kind of browser, the languages it speaks, etc. That's helpful for marketers to track users, but it's a bit over the top for small microcontrollers with limited memory and processing power. 

We'll check out more efficient means for microcontrollers to communicate over the internet. 

## Secure Client

The secure, https, client is almost the same except that the port has been changed from 80 to 443 and the line `s = ssl.wrap_socket(s)` been added.

In [1]:
%connect esp32
%softreset

import socket, ssl

ai = socket.getaddrinfo("google.com", 443)
print("Address information:", ai)
addr = ai[0][-1]

s = socket.socket()
s.connect(addr)
s = ssl.wrap_socket(s)

s.write(b"GET / HTTP/1.0\r\n\r\n")

print("\nResponse:")
print(s.read(2000).decode())

s.close()

Connected to esp32 @ serial:///dev/ttyUSB0

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!!!!   softreset ...     !!!!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

Address information: [(2, 1, 0, 'google.com', ('216.58.194.174', 443))]

Response:
HTTP/1.0 200 OK
Date: Thu, 22 Jul 2021 03:05:33 GMT
Expires: -1
Cache-Control: private, max-age=0
Content-Type: text/html; charset=ISO-8859-1
P3P: CP="This is not a P3P policy! See g.co/p3phelp for more info."
Server: gws
X-XSS-Protection: 0
X-Frame-Options: SAMEORIGIN
Set-Cookie: 1P_JAR=2021-07-22-03; expires=Sat, 21-Aug-2021 03:05:33 GMT; path=/; domain=.google.com; Secure
Set-Cookie: NID=219=rjuTN3Kqr-3ZPMHKtq8jQjursFOga8OXMutFBj2WQzFgu6RWcH3siOduyS4k8FdDltMeoieIF612FtjMk1FeHPxhHzonnAw1J8LFjOcg7N4J45x9oBw2i5d4xrlcaN55HX_4JJXOI8hqNt5GRTSf8AXtMKpm_OGirlei0Fca3IU; expires=Fri, 21-Jan-2022 03:05:33 GMT; path=/; domain=.google.com; HttpOnly
Alt-Svc: h3=":443"; ma=2592000,h3-29=":443"; ma=2592000,h3-T051=":443"; ma=2592000,h3-Q050=":443"; ma=2592000,h3-Q046=":443"; ma=

## Secure Server

* MicroPython ssl broken? Apparently a memory issue:
    * [Github issue](https://github.com/micropython/micropython/issues/5543)
    * [Forum Post](https://forum.micropython.org/viewtopic.php?f=18&t=10375&hilit=cert)
* [CPython Tutorial](https://realpython.com/python-https/)

Secure webservers use a certificate and a private key to encrypt data and "prove" their identity to the web client (e.g. browser).

First we find the IP address of the microcontroller and save it in the jupyter store to access it later from bash.

In [9]:
# get IP and save in the Jupyter store
import network
my_ip = network.WLAN(network.STA_IF).ifconfig()[0]
%store my_ip

# retrieve it on the host (CPython) and assign it to a shell environment variable
%%host
%store -r my_ip
os.environ["my_ip"] = my_ip

# verify that bash has the correct IP
%%bash
echo "microcontroller IP:" $my_ip

microcontroller IP: 10.39.40.135


Now we create the "configuration" from which the certificate and key will be generated. Change the values in the `[req_distinguished_name]` section if you wish (the defaults are ok).

In [ ]:
%cd $IOT_PROJECTS/internet
!mkdir -p ssl
%cd ssl

In [12]:
%%bash
cat << EOF >cert.conf
[req]
distinguished_name = req_distinguished_name
x509_extensions = v3_req
prompt = no
[req_distinguished_name]
C = US
ST = CA
L = San Francisco
O = MicroPython Webserver
OU = iot49
CN = iot49
[v3_req]
keyUsage = critical, digitalSignature, keyAgreement
extendedKeyUsage = serverAuth
subjectAltName = @alt_names
[alt_names]
DNS.1 = $my_ip
IP.1  = $my_ip
EOF

cwd = /home/iot/iot49.org/docs/projects/internet
cwd = /home/iot/iot49.org/docs/projects/internet/ssl
cert.conf


The next step is to generate the key and cert files from the configuration.

In [12]:
%%bash

openssl req -x509 -nodes -days 3650 -newkey rsa:2048 \
    -keyout cert.key -out cert.crt -config cert.conf       

Generating a RSA private key
.........+++++
.....................+++++
writing new private key to 'cert.key'
-----
total 12
-rw-r--r-- 1 iot iot  366 Jul 22 09:58 cert.conf
-rw-r--r-- 1 iot iot 1350 Jul 22 10:00 cert.crt
-rw------- 1 iot iot 1704 Jul 22 10:00 cert.key
